In [12]:
import mysql.connector as connection
import pandas as pd
import numpy as np
import pickle

In [2]:
# Establish connection with MySQL DB
mydb = connection.connect(host="localhost", port="3306", user="root", password="", database="food_ordering_db")

In [3]:
# Accessing food table
query = """Select *
						FROM `food`;"""
food = pd.read_sql(query,mydb)
print(food.head())

# Accessing user table
query = """Select *
						FROM `user`;"""
users = pd.read_sql(query,mydb)
print(users.head())

   foodID  restaurantID                      foodName  price
0       1             1  Mala Grilled Burmese Chicken  10000
1       2             1             Mala French Fries   3500
2       3             1         Rakhine Seafood Salad   6000
3       4             1           Steamed Chicken Leg   5000
4       5             1            Mala Grilled Prawn  12000
   userID firstName lastName
0       1     James    Smith
1       2   Michael    Smith
2       3    Robert    Smith
3       4     David    Smith
4       5     James  Johnson


In [4]:
# Accessing userID and foodID data with ratings
query = """Select o.foodorderID, o.foodID, o.rating, c.userID 
						FROM `foodorder` o, `cart` c
						WHERE o.cartID = c.cartID;"""
orders = pd.read_sql(query,mydb)
print(orders.head())
print(list(orders.columns))
print("No. of orders - ", len(orders))

   orderID  foodID  rating  userID
0        1     283       3     166
1        2    6742       4     201
2        3    3669      10     225
3        4    3910       2      25
4        5     291       3     166
['orderID', 'foodID', 'rating', 'userID']
No. of orders -  300


In [5]:
# Merge order and food queries
combine_food_order = pd.merge(orders, food, on="foodID")
print(combine_food_order.head())
print("No. of combine_food_order - ", len(combine_food_order))

   orderID  foodID  rating  userID  restaurantID  \
0        1     283       3     166             7   
1        2    6742       4     201           219   
2        3    3669      10     225           121   
3       33    3669       2     225           121   
4        4    3910       2      25           129   

                        foodName  price  
0  Sour and Spicy Crab Meat Soup   4500  
1                  Yakhine Salad    800  
2            Plain Water Spinach   1200  
3            Plain Water Spinach   1200  
4                    Kiwi Yogurt   1800  
No. of combine_food_order -  300


In [6]:
print(f'Amount of unique foodIDs - {len(pd.unique(combine_food_order["foodID"]))}')
print(f'Amount of unique userIDs - {len(pd.unique(combine_food_order["userID"]))}')

Amount of unique foodIDs - 245
Amount of unique userIDs - 48


In [7]:
with open("combined_food_order" + ".csv", "w") as f:
	f.write(combine_food_order.to_csv())

Getting the number of times food was ordered

In [8]:
# combine_food_order = combine_food_order.dropna(axis = 0, subset = ['bookTitle'])
food_order_count = (combine_food_order.
     groupby(by = ["foodID"]).
     count().
     reset_index().
     sort_values(by = ["orderID"]).
     rename(columns = {"orderID": "totalCount"})
     [["foodID", "totalCount"]]
     )
food_order_count.tail()

,foodID,totalCount
178,5173,4
137,4026,4
123,3675,4
162,4917,4
223,6908,6


In [9]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(food_order_count["totalCount"].describe())

count   245.000
mean      1.224
std       0.661
min       1.000
25%       1.000
50%       1.000
75%       1.000
max       6.000
Name: totalCount, dtype: float64


In [10]:
order_with_total_count = combine_food_order.merge(food_order_count, left_on = 'foodID', right_on = 'foodID', how = 'left')
# order_with_total_count["has_ordered"] = 1
print(order_with_total_count.loc[order_with_total_count["foodName"] == "Noodle Salad"])
# order_with_total_count.head()
print("No. of order_with_total_count - ", len(order_with_total_count))

     orderID  foodID  rating  userID  restaurantID      foodName  price  \
282      271    4948       6     115           167  Noodle Salad   1200   
288      282    3144       4     218           106  Noodle Salad    700   

     totalCount  
282           1  
288           1  
No. of order_with_total_count -  300


In [57]:
with open("order_with_total_count" + ".csv", "w") as f:
	f.write(order_with_total_count.to_csv())

In [17]:
from scipy.sparse import csr_matrix
order_with_total_count = order_with_total_count.drop_duplicates(['userID', 'foodName'])
# order_with_total_count_pivot = order_with_total_count.pivot(index = 'foodName', columns = 'userID', values = 'has_ordered').fillna(0)
order_with_total_count_pivot = order_with_total_count.pivot(index = 'foodName', columns = 'userID', values = 'rating').fillna(0)
order_with_total_count_foodID_pivot = order_with_total_count.pivot(index = 'foodID', columns = 'userID', values = 'rating').fillna(0)
order_with_total_count_matrix = csr_matrix(order_with_total_count_pivot.values)

from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(order_with_total_count_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [18]:
# Its important to use binary mode 
knnPickle = open('knnpickle_file', 'wb') 
# source, destination 
pickle.dump(model_knn, knnPickle)  
# close the file
knnPickle.close()

In [56]:
with open("order_with_total_count_pivot" + ".csv", "w") as f:
	f.write(order_with_total_count_pivot.to_csv())
with open("order_with_total_count_pivot_id" + ".csv", "w") as f:
	f.write(order_with_total_count_foodID_pivot.to_csv())
with open("query_index_to_foodID" + ".csv", "w") as f:
	f.write("query_index,foodID\n")
	pivot_foodID_list = list(order_with_total_count_foodID_pivot.index)
	for i in range(len(pivot_foodID_list)):
		f.write(f"{i},{pivot_foodID_list[i]}\n")

In [23]:
# query_index = np.random.choice(order_with_total_count_pivot.shape[0])
# print(query_index)
# query_index = 124
query_index = 164		#(points to foodID 4924 Real Milk)
distances, indices = model_knn.kneighbors(order_with_total_count_foodID_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

order_with_total_count_foodID_pivot.iloc[query_index,:].values.reshape(1,-1)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 6., 0.,
        0., 0., 0., 0., 0., 0., 6., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [31]:
foodID = order_with_total_count_foodID_pivot.index[query_index]
foodName = food.foodName[foodID-1]
print(foodID)
print(foodName)

4924
Real Milk


In [35]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(foodName))
    else:
        foodID = order_with_total_count_foodID_pivot.index[indices.flatten()[i]]
        foodName = food.foodName[foodID-1]
        print('{0}: {1}, with distance of {2}:'.format(i, foodName, distances.flatten()[i]))

Recommendations for Real Milk:

1: Mandalay Mixed Fruits Yogurt, with distance of 0.10557280900008403:
2: Mandalay Shan Noodle with Sticky Shan Noodle, with distance of 0.21064778262367367:
3: Mandalay Rice Noodle, with distance of 0.2928932188134524:
4: French Toast, with distance of 0.2928932188134524:
5: Bread with Malai, with distance of 0.2928932188134524:
